In [11]:
!pip install optuna

In [12]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.metrics import accuracy_score, matthews_corrcoef, make_scorer
import optuna
from deeptlf import DeepTFL

In [13]:
# This is a function that loads training and testing data for 10-fold cross validation
def prepare_train_test_dataset(datasets, test_idx):
  test_df = datasets[test_idx]
  train_df = pd.concat([datasets[i] for i in range(len(datasets)) if i != test_idx])

  X_train = train_df.iloc[:, 1:-1].values
  X_test = test_df.iloc[:, 1:-1].values
  y_train = train_df.iloc[:, -1].values
  y_test = test_df.iloc[:, -1].values

  return X_train, X_test, y_train, y_test

In [14]:
import torch
import time

def train_with_base_deeptlf(datasets, key):
  start_time = time.time()
  print("Currently training base DeepTLF model with dataset key:", key)

  # Create a Pandas DataFrame to store all experiment results
  results_df = pd.DataFrame(columns=['Fold', 'Accuracy', 'MCC'])

  # Perform 10-fold cross validation
  dataset_list = datasets[key]

  for i in range(10):
    X_train, X_test, y_train, y_test = prepare_train_test_dataset(dataset_list, i)

    # Train the model
    model = DeepTFL()
    model.fit(X_train=X_train, y_train=y_train, X_val=X_test, y_val=y_test)

    # Evaluate the model
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    mcc = matthews_corrcoef(y_test, y_pred)

    print("Fold", i+1, "Accuracy:", accuracy, "MCC:", mcc)
    results_df.loc[i] = [i+1, accuracy, mcc]

  elapsed_time = time.time() - start_time
  print(f"Elapsed time: {elapsed_time:.2f} seconds")

  return results_df

In [15]:
def create_objective_for_deeptlf(X_train, X_test, y_train, y_test):
  def objective(trial):
      params = {
          "n_est": trial.suggest_int("n_est", 10, 50, step=5),
          "max_depth": trial.suggest_int("max_depth", 3, 10),
          "drop": trial.suggest_float("drop", 0.1, 0.5, step=0.05),
          "hidden_dim": trial.suggest_int("hidden_dim", 64, 512, step=64),
          "n_layers": trial.suggest_int("n_layers", 2, 6),
          "xgb_lr": trial.suggest_float("xgb_lr", 0.01, 0.5, step=0.01),
      }

      model = DeepTFL(
          n_est=params["n_est"],
          max_depth=params["max_depth"],
          drop=params["drop"],
          hidden_dim=params["hidden_dim"],
          n_layers=params["n_layers"],
          xgb_lr=params["xgb_lr"],
          task="class"
      )
      model.fit(X_train=X_train, y_train=y_train, X_val=X_test, y_val=y_test)

      y_pred = model.predict(X_test)
      mcc = matthews_corrcoef(y_test, y_pred)

      # Return the accuracy
      return mcc

  return objective

In [16]:
def train_with_best_hyperparameters(datasets, key):
  start_time = time.time()
  print("Currently finetuning DeepTLF model with dataset key:", key)

  # Create a Pandas DataFrame to store all experiment results
  results_df = pd.DataFrame(columns=['Fold', 'Accuracy', 'MCC'])

  # Perform 10-fold cross validation
  dataset_list = datasets[key]

  # Use the first data as the test for the HPO
  X_train, X_test, y_train, y_test = prepare_train_test_dataset(dataset_list, 0)

  objective_function = create_objective_for_deeptlf(X_train, X_test, y_train, y_test)

  study = optuna.create_study(direction='maximize')
  study.optimize(objective_function, n_trials=15, gc_after_trial=True)

  best_params = study.best_params
  print("Best hyperparameters:", best_params)

  # Use best hyperparams to conduct a 10-fold cross validation
  for i in range(10):
    X_train, X_test, y_train, y_test = prepare_train_test_dataset(dataset_list, i)

    model = DeepTFL(
        n_est=best_params["n_est"],
        max_depth=best_params["max_depth"],
        drop=best_params["drop"],
        hidden_dim=best_params["hidden_dim"],
        n_layers=best_params["n_layers"],
        xgb_lr=best_params["xgb_lr"],
        task="class"
    )
    model.fit(X_train=X_train, y_train=y_train, X_val=X_test, y_val=y_test)

    # Evaluate the model
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    mcc = matthews_corrcoef(y_test, y_pred)

    print("Fold", i+1, "Accuracy:", accuracy, "MCC:", mcc)
    results_df.loc[i] = [i+1, accuracy, mcc]

  elapsed_time = time.time() - start_time
  print(f"Elapsed time: {elapsed_time:.2f} seconds")

  return results_df

In [17]:
# Load the validation datasets

datasets = {}

# All

XII_all_nopcc_1 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/all/cv/nopcc/fold_1.csv')
XII_all_nopcc_2 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/all/cv/nopcc/fold_2.csv')
XII_all_nopcc_3 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/all/cv/nopcc/fold_3.csv')
XII_all_nopcc_4 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/all/cv/nopcc/fold_4.csv')
XII_all_nopcc_5 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/all/cv/nopcc/fold_5.csv')
XII_all_nopcc_6 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/all/cv/nopcc/fold_6.csv')
XII_all_nopcc_7 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/all/cv/nopcc/fold_7.csv')
XII_all_nopcc_8 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/all/cv/nopcc/fold_8.csv')
XII_all_nopcc_9 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/all/cv/nopcc/fold_9.csv')
XII_all_nopcc_10 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/all/cv/nopcc/fold_10.csv')
datasets['XII_all_nopcc'] = [XII_all_nopcc_1, XII_all_nopcc_2, XII_all_nopcc_3, XII_all_nopcc_4, XII_all_nopcc_5,
                                       XII_all_nopcc_6, XII_all_nopcc_7, XII_all_nopcc_8, XII_all_nopcc_9, XII_all_nopcc_10]

XII_all_pcc95_1 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/all/cv/pcc95/fold_1.csv')
XII_all_pcc95_2 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/all/cv/pcc95/fold_2.csv')
XII_all_pcc95_3 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/all/cv/pcc95/fold_3.csv')
XII_all_pcc95_4 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/all/cv/pcc95/fold_4.csv')
XII_all_pcc95_5 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/all/cv/pcc95/fold_5.csv')
XII_all_pcc95_6 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/all/cv/pcc95/fold_6.csv')
XII_all_pcc95_7 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/all/cv/pcc95/fold_7.csv')
XII_all_pcc95_8 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/all/cv/pcc95/fold_8.csv')
XII_all_pcc95_9 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/all/cv/pcc95/fold_9.csv')
XII_all_pcc95_10 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/all/cv/pcc95/fold_10.csv')
datasets['XII_all_pcc95'] = [XII_all_pcc95_1, XII_all_pcc95_2, XII_all_pcc95_3, XII_all_pcc95_4, XII_all_pcc95_5,
                                       XII_all_pcc95_6, XII_all_pcc95_7, XII_all_pcc95_8, XII_all_pcc95_9, XII_all_pcc95_10]

XII_all_pcc75_1 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/all/cv/pcc75/fold_1.csv')
XII_all_pcc75_2 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/all/cv/pcc75/fold_2.csv')
XII_all_pcc75_3 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/all/cv/pcc75/fold_3.csv')
XII_all_pcc75_4 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/all/cv/pcc75/fold_4.csv')
XII_all_pcc75_5 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/all/cv/pcc75/fold_5.csv')
XII_all_pcc75_6 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/all/cv/pcc75/fold_6.csv')
XII_all_pcc75_7 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/all/cv/pcc75/fold_7.csv')
XII_all_pcc75_8 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/all/cv/pcc75/fold_8.csv')
XII_all_pcc75_9 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/all/cv/pcc75/fold_9.csv')
XII_all_pcc75_10 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/all/cv/pcc75/fold_10.csv')
datasets['XII_all_pcc75'] = [XII_all_pcc75_1, XII_all_pcc75_2, XII_all_pcc75_3, XII_all_pcc75_4, XII_all_pcc75_5,
                                       XII_all_pcc75_6, XII_all_pcc75_7, XII_all_pcc75_8, XII_all_pcc75_9, XII_all_pcc75_10]

# 200

XII_200_nopcc_1 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/200/cv/nopcc/fold_1.csv')
XII_200_nopcc_2 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/200/cv/nopcc/fold_2.csv')
XII_200_nopcc_3 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/200/cv/nopcc/fold_3.csv')
XII_200_nopcc_4 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/200/cv/nopcc/fold_4.csv')
XII_200_nopcc_5 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/200/cv/nopcc/fold_5.csv')
XII_200_nopcc_6 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/200/cv/nopcc/fold_6.csv')
XII_200_nopcc_7 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/200/cv/nopcc/fold_7.csv')
XII_200_nopcc_8 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/200/cv/nopcc/fold_8.csv')
XII_200_nopcc_9 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/200/cv/nopcc/fold_9.csv')
XII_200_nopcc_10 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/200/cv/nopcc/fold_10.csv')
datasets['XII_200_nopcc'] = [XII_200_nopcc_1, XII_200_nopcc_2, XII_200_nopcc_3, XII_200_nopcc_4, XII_200_nopcc_5,
                                       XII_200_nopcc_6, XII_200_nopcc_7, XII_200_nopcc_8, XII_200_nopcc_9, XII_200_nopcc_10]

XII_200_pcc95_1 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/200/cv/pcc95/fold_1.csv')
XII_200_pcc95_2 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/200/cv/pcc95/fold_2.csv')
XII_200_pcc95_3 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/200/cv/pcc95/fold_3.csv')
XII_200_pcc95_4 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/200/cv/pcc95/fold_4.csv')
XII_200_pcc95_5 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/200/cv/pcc95/fold_5.csv')
XII_200_pcc95_6 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/200/cv/pcc95/fold_6.csv')
XII_200_pcc95_7 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/200/cv/pcc95/fold_7.csv')
XII_200_pcc95_8 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/200/cv/pcc95/fold_8.csv')
XII_200_pcc95_9 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/200/cv/pcc95/fold_9.csv')
XII_200_pcc95_10 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/200/cv/pcc95/fold_10.csv')
datasets['XII_200_pcc95'] = [XII_200_pcc95_1, XII_200_pcc95_2, XII_200_pcc95_3, XII_200_pcc95_4, XII_200_pcc95_5,
                                       XII_200_pcc95_6, XII_200_pcc95_7, XII_200_pcc95_8, XII_200_pcc95_9, XII_200_pcc95_10]

XII_200_pcc75_1 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/200/cv/pcc75/fold_1.csv')
XII_200_pcc75_2 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/200/cv/pcc75/fold_2.csv')
XII_200_pcc75_3 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/200/cv/pcc75/fold_3.csv')
XII_200_pcc75_4 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/200/cv/pcc75/fold_4.csv')
XII_200_pcc75_5 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/200/cv/pcc75/fold_5.csv')
XII_200_pcc75_6 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/200/cv/pcc75/fold_6.csv')
XII_200_pcc75_7 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/200/cv/pcc75/fold_7.csv')
XII_200_pcc75_8 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/200/cv/pcc75/fold_8.csv')
XII_200_pcc75_9 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/200/cv/pcc75/fold_9.csv')
XII_200_pcc75_10 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/200/cv/pcc75/fold_10.csv')
datasets['XII_200_pcc75'] = [XII_200_pcc75_1, XII_200_pcc75_2, XII_200_pcc75_3, XII_200_pcc75_4, XII_200_pcc75_5,
                                       XII_200_pcc75_6, XII_200_pcc75_7, XII_200_pcc75_8, XII_200_pcc75_9, XII_200_pcc75_10]

# 150

XII_150_nopcc_1 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/150/cv/nopcc/fold_1.csv')
XII_150_nopcc_2 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/150/cv/nopcc/fold_2.csv')
XII_150_nopcc_3 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/150/cv/nopcc/fold_3.csv')
XII_150_nopcc_4 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/150/cv/nopcc/fold_4.csv')
XII_150_nopcc_5 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/150/cv/nopcc/fold_5.csv')
XII_150_nopcc_6 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/150/cv/nopcc/fold_6.csv')
XII_150_nopcc_7 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/150/cv/nopcc/fold_7.csv')
XII_150_nopcc_8 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/150/cv/nopcc/fold_8.csv')
XII_150_nopcc_9 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/150/cv/nopcc/fold_9.csv')
XII_150_nopcc_10 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/150/cv/nopcc/fold_10.csv')
datasets['XII_150_nopcc'] = [XII_150_nopcc_1, XII_150_nopcc_2, XII_150_nopcc_3, XII_150_nopcc_4, XII_150_nopcc_5,
                                       XII_150_nopcc_6, XII_150_nopcc_7, XII_150_nopcc_8, XII_150_nopcc_9, XII_150_nopcc_10]

XII_150_pcc95_1 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/150/cv/pcc95/fold_1.csv')
XII_150_pcc95_2 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/150/cv/pcc95/fold_2.csv')
XII_150_pcc95_3 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/150/cv/pcc95/fold_3.csv')
XII_150_pcc95_4 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/150/cv/pcc95/fold_4.csv')
XII_150_pcc95_5 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/150/cv/pcc95/fold_5.csv')
XII_150_pcc95_6 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/150/cv/pcc95/fold_6.csv')
XII_150_pcc95_7 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/150/cv/pcc95/fold_7.csv')
XII_150_pcc95_8 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/150/cv/pcc95/fold_8.csv')
XII_150_pcc95_9 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/150/cv/pcc95/fold_9.csv')
XII_150_pcc95_10 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/150/cv/pcc95/fold_10.csv')
datasets['XII_150_pcc95'] = [XII_150_pcc95_1, XII_150_pcc95_2, XII_150_pcc95_3, XII_150_pcc95_4, XII_150_pcc95_5,
                                       XII_150_pcc95_6, XII_150_pcc95_7, XII_150_pcc95_8, XII_150_pcc95_9, XII_150_pcc95_10]

XII_150_pcc75_1 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/150/cv/pcc75/fold_1.csv')
XII_150_pcc75_2 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/150/cv/pcc75/fold_2.csv')
XII_150_pcc75_3 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/150/cv/pcc75/fold_3.csv')
XII_150_pcc75_4 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/150/cv/pcc75/fold_4.csv')
XII_150_pcc75_5 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/150/cv/pcc75/fold_5.csv')
XII_150_pcc75_6 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/150/cv/pcc75/fold_6.csv')
XII_150_pcc75_7 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/150/cv/pcc75/fold_7.csv')
XII_150_pcc75_8 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/150/cv/pcc75/fold_8.csv')
XII_150_pcc75_9 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/150/cv/pcc75/fold_9.csv')
XII_150_pcc75_10 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/150/cv/pcc75/fold_10.csv')
datasets['XII_150_pcc75'] = [XII_150_pcc75_1, XII_150_pcc75_2, XII_150_pcc75_3, XII_150_pcc75_4, XII_150_pcc75_5,
                                       XII_150_pcc75_6, XII_150_pcc75_7, XII_150_pcc75_8, XII_150_pcc75_9, XII_150_pcc75_10]

# 100

XII_100_nopcc_1 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/100/cv/nopcc/fold_1.csv')
XII_100_nopcc_2 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/100/cv/nopcc/fold_2.csv')
XII_100_nopcc_3 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/100/cv/nopcc/fold_3.csv')
XII_100_nopcc_4 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/100/cv/nopcc/fold_4.csv')
XII_100_nopcc_5 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/100/cv/nopcc/fold_5.csv')
XII_100_nopcc_6 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/100/cv/nopcc/fold_6.csv')
XII_100_nopcc_7 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/100/cv/nopcc/fold_7.csv')
XII_100_nopcc_8 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/100/cv/nopcc/fold_8.csv')
XII_100_nopcc_9 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/100/cv/nopcc/fold_9.csv')
XII_100_nopcc_10 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/100/cv/nopcc/fold_10.csv')
datasets['XII_100_nopcc'] = [XII_100_nopcc_1, XII_100_nopcc_2, XII_100_nopcc_3, XII_100_nopcc_4, XII_100_nopcc_5,
                                       XII_100_nopcc_6, XII_100_nopcc_7, XII_100_nopcc_8, XII_100_nopcc_9, XII_100_nopcc_10]

XII_100_pcc95_1 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/100/cv/pcc95/fold_1.csv')
XII_100_pcc95_2 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/100/cv/pcc95/fold_2.csv')
XII_100_pcc95_3 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/100/cv/pcc95/fold_3.csv')
XII_100_pcc95_4 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/100/cv/pcc95/fold_4.csv')
XII_100_pcc95_5 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/100/cv/pcc95/fold_5.csv')
XII_100_pcc95_6 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/100/cv/pcc95/fold_6.csv')
XII_100_pcc95_7 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/100/cv/pcc95/fold_7.csv')
XII_100_pcc95_8 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/100/cv/pcc95/fold_8.csv')
XII_100_pcc95_9 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/100/cv/pcc95/fold_9.csv')
XII_100_pcc95_10 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/100/cv/pcc95/fold_10.csv')
datasets['XII_100_pcc95'] = [XII_100_pcc95_1, XII_100_pcc95_2, XII_100_pcc95_3, XII_100_pcc95_4, XII_100_pcc95_5,
                                       XII_100_pcc95_6, XII_100_pcc95_7, XII_100_pcc95_8, XII_100_pcc95_9, XII_100_pcc95_10]

XII_100_pcc75_1 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/100/cv/pcc75/fold_1.csv')
XII_100_pcc75_2 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/100/cv/pcc75/fold_2.csv')
XII_100_pcc75_3 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/100/cv/pcc75/fold_3.csv')
XII_100_pcc75_4 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/100/cv/pcc75/fold_4.csv')
XII_100_pcc75_5 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/100/cv/pcc75/fold_5.csv')
XII_100_pcc75_6 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/100/cv/pcc75/fold_6.csv')
XII_100_pcc75_7 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/100/cv/pcc75/fold_7.csv')
XII_100_pcc75_8 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/100/cv/pcc75/fold_8.csv')
XII_100_pcc75_9 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/100/cv/pcc75/fold_9.csv')
XII_100_pcc75_10 = pd.read_csv('/content/drive/MyDrive/OrderedDataset/XII/100/cv/pcc75/fold_10.csv')
datasets['XII_100_pcc75'] = [XII_100_pcc75_1, XII_100_pcc75_2, XII_100_pcc75_3, XII_100_pcc75_4, XII_100_pcc75_5,
                                       XII_100_pcc75_6, XII_100_pcc75_7, XII_100_pcc75_8, XII_100_pcc75_9, XII_100_pcc75_10]

In [18]:
def view_results_df(results_df, key):
    print("10-fold cross validation results for results_df with key:", key)
    for i in range(len(results_df)):
        # Format accuracy and MCC to 3 decimal places
        accuracy = f"{results_df['Accuracy'][i]:.3f}"
        mcc = f"{results_df['MCC'][i]:.3f}"
        print(f"Fold {i}, Accuracy: {accuracy}, MCC: {mcc}")


In [19]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [20]:
XII_all_nopcc_results_df = train_with_base_deeptlf(datasets, 'XII_all_nopcc')
print("\n---------------------------------------------------------------------\n")

Currently training base DeepTLF model with dataset key: XII_all_nopcc


Epochs:  35%|███▌      | 35/100 [00:02<00:04, 15.03it/s]


Early stopping
Fold 1 Accuracy: 0.8775510204081632 MCC: 0.7645166003324636


Epochs:  37%|███▋      | 37/100 [00:01<00:03, 18.89it/s]


Early stopping
Fold 2 Accuracy: 0.8979591836734694 MCC: 0.7966666666666666


...

Early stopping
Fold 8 Accuracy: 0.875 MCC: 0.7526178090063816


Epochs:  36%|███▌      | 36/100 [00:02<00:03, 16.91it/s]


Early stopping
Fold 9 Accuracy: 0.8125 MCC: 0.6255432421712243


Epochs:  32%|███▏      | 32/100 [00:01<00:03, 18.24it/s]


Early stopping
Fold 10 Accuracy: 0.7916666666666666 MCC: 0.5853694070049635
Elapsed time: 46.98 seconds

---------------------------------------------------------------------



In [21]:

XII_all_pcc95_results_df = train_with_base_deeptlf(datasets, 'XII_all_pcc95')
print("\n---------------------------------------------------------------------\n")


Currently training base DeepTLF model with dataset key: XII_all_pcc95


Epochs:  32%|███▏      | 32/100 [00:01<00:02, 31.15it/s]


Early stopping
Fold 1 Accuracy: 0.8979591836734694 MCC: 0.7966666666666666


Epochs:  32%|███▏      | 32/100 [00:01<00:02, 30.11it/s]


Early stopping
Fold 2 Accuracy: 0.8775510204081632 MCC: 0.7571284874952731


...

Early stopping
Fold 8 Accuracy: 0.875 MCC: 0.75


Epochs:  48%|████▊     | 48/100 [00:01<00:01, 33.53it/s]


Early stopping
Fold 9 Accuracy: 0.875 MCC: 0.75


Epochs:  21%|██        | 21/100 [00:00<00:02, 33.76it/s]


Early stopping
Fold 10 Accuracy: 0.5 MCC: 0.0
Elapsed time: 23.03 seconds

---------------------------------------------------------------------



In [22]:
XII_all_pcc75_results_df = train_with_base_deeptlf(datasets, 'XII_all_pcc75')

Currently training base DeepTLF model with dataset key: XII_all_pcc75


Epochs:  26%|██▌       | 26/100 [00:00<00:01, 42.17it/s]


Early stopping
Fold 1 Accuracy: 0.7755102040816326 MCC: 0.5772364433999367


Epochs:  34%|███▍      | 34/100 [00:00<00:01, 40.94it/s]


Early stopping
Fold 2 Accuracy: 0.9183673469387755 MCC: 0.8397340583424286


...

Early stopping
Fold 8 Accuracy: 0.8958333333333334 MCC: 0.7923547734168841


Epochs:  38%|███▊      | 38/100 [00:00<00:01, 44.63it/s]


Early stopping
Fold 9 Accuracy: 0.8541666666666666 MCC: 0.7139328934618737


Epochs:  23%|██▎       | 23/100 [00:00<00:01, 46.59it/s]


Early stopping
Fold 10 Accuracy: 0.5625 MCC: 0.1487044791289829
Elapsed time: 12.97 seconds


In [23]:

print("\n---------------------------------------------------------------------\n")
XII_200_nopcc_results_df = train_with_base_deeptlf(datasets, 'XII_200_nopcc')


---------------------------------------------------------------------

Currently training base DeepTLF model with dataset key: XII_200_nopcc


Epochs:  35%|███▌      | 35/100 [00:02<00:03, 17.36it/s]


Early stopping
Fold 1 Accuracy: 0.875 MCC: 0.7509392614826383


Epochs:  31%|███       | 31/100 [00:01<00:04, 16.48it/s]


Early stopping
Fold 2 Accuracy: 0.95 MCC: 0.9


...

Early stopping
Fold 8 Accuracy: 0.875 MCC: 0.7509392614826383


Epochs:  47%|████▋     | 47/100 [00:02<00:03, 16.71it/s]


Early stopping
Fold 9 Accuracy: 0.925 MCC: 0.8510644963469901


Epochs:  42%|████▏     | 42/100 [00:02<00:03, 17.94it/s]


Early stopping
Fold 10 Accuracy: 0.8974358974358975 MCC: 0.7994750784123957
Elapsed time: 42.68 seconds


In [24]:

print("\n---------------------------------------------------------------------\n")
XII_200_pcc95_results_df = train_with_base_deeptlf(datasets, 'XII_200_pcc95')


---------------------------------------------------------------------

Currently training base DeepTLF model with dataset key: XII_200_pcc95


Epochs:  30%|███       | 30/100 [00:01<00:02, 28.05it/s]


Early stopping
Fold 1 Accuracy: 0.85 MCC: 0.7144345083117603


Epochs:  37%|███▋      | 37/100 [00:01<00:02, 31.29it/s]


Early stopping
Fold 2 Accuracy: 0.95 MCC: 0.9


...

Early stopping
Fold 8 Accuracy: 0.9 MCC: 0.8


Epochs:  67%|██████▋   | 67/100 [00:01<00:00, 35.52it/s]


Early stopping
Fold 9 Accuracy: 1.0 MCC: 1.0


Epochs:  36%|███▌      | 36/100 [00:00<00:01, 38.32it/s]


Early stopping
Fold 10 Accuracy: 0.9487179487179487 MCC: 0.9023778112773576
Elapsed time: 22.85 seconds


In [25]:

print("\n---------------------------------------------------------------------\n")
XII_200_pcc75_results_df = train_with_base_deeptlf(datasets, 'XII_200_pcc75')


---------------------------------------------------------------------

Currently training base DeepTLF model with dataset key: XII_200_pcc75


Epochs:  29%|██▉       | 29/100 [00:00<00:01, 44.03it/s]


Early stopping
Fold 1 Accuracy: 0.925 MCC: 0.8510644963469901


Epochs:  60%|██████    | 60/100 [00:01<00:00, 47.23it/s]


Early stopping
Fold 2 Accuracy: 0.95 MCC: 0.9


...

Early stopping
Fold 8 Accuracy: 0.875 MCC: 0.7509392614826383


Epochs:  33%|███▎      | 33/100 [00:00<00:01, 47.26it/s]


Early stopping
Fold 9 Accuracy: 0.925 MCC: 0.8510644963469901


Epochs:  44%|████▍     | 44/100 [00:00<00:01, 48.09it/s]

Early stopping
Fold 10 Accuracy: 0.8974358974358975 MCC: 0.7947368421052632
Elapsed time: 12.59 seconds


In [26]:

print("\n---------------------------------------------------------------------\n")
XII_150_nopcc_results_df = train_with_base_deeptlf(datasets, 'XII_150_nopcc')


---------------------------------------------------------------------

Currently training base DeepTLF model with dataset key: XII_150_nopcc


Epochs:  43%|████▎     | 43/100 [00:01<00:01, 32.77it/s]


Early stopping
Fold 1 Accuracy: 0.8666666666666667 MCC: 0.7333333333333333


Epochs:  42%|████▏     | 42/100 [00:01<00:02, 25.55it/s]


Early stopping
Fold 2 Accuracy: 0.9 MCC: 0.8017837257372731


...

Early stopping
Fold 8 Accuracy: 0.9 MCC: 0.8017837257372731


Epochs:  45%|████▌     | 45/100 [00:01<00:01, 34.74it/s]


Early stopping
Fold 9 Accuracy: 1.0 MCC: 1.0


Epochs:  35%|███▌      | 35/100 [00:01<00:02, 29.26it/s]


Early stopping
Fold 10 Accuracy: 0.896551724137931 MCC: 0.809173593712687
Elapsed time: 36.10 seconds


In [27]:

print("\n---------------------------------------------------------------------\n")
XII_150_pcc95_results_df = train_with_base_deeptlf(datasets, 'XII_150_pcc95')


---------------------------------------------------------------------

Currently training base DeepTLF model with dataset key: XII_150_pcc95


Epochs:  36%|███▌      | 36/100 [00:01<00:01, 35.18it/s]


Early stopping
Fold 1 Accuracy: 0.8666666666666667 MCC: 0.7399400733959437


Epochs:  78%|███████▊  | 78/100 [00:01<00:00, 42.07it/s]


Early stopping
Fold 2 Accuracy: 0.9333333333333333 MCC: 0.8666666666666667


...

Early stopping
Fold 8 Accuracy: 1.0 MCC: 1.0


Epochs:  49%|████▉     | 49/100 [00:01<00:01, 34.41it/s]


Early stopping
Fold 9 Accuracy: 1.0 MCC: 1.0


Epochs:  39%|███▉      | 39/100 [00:00<00:01, 39.43it/s]


Early stopping
Fold 10 Accuracy: 0.9310344827586207 MCC: 0.8696565534786727
Elapsed time: 21.65 seconds


In [28]:

print("\n---------------------------------------------------------------------\n")
XII_150_pcc75_results_df = train_with_base_deeptlf(datasets, 'XII_150_pcc75')


---------------------------------------------------------------------

Currently training base DeepTLF model with dataset key: XII_150_pcc75


Epochs:  43%|████▎     | 43/100 [00:00<00:00, 59.86it/s]


Early stopping
Fold 1 Accuracy: 0.9 MCC: 0.8017837257372731


Epochs:  47%|████▋     | 47/100 [00:00<00:00, 62.25it/s]


Early stopping
Fold 2 Accuracy: 0.9333333333333333 MCC: 0.8666666666666667


...

Early stopping
Fold 8 Accuracy: 1.0 MCC: 1.0


Epochs:  78%|███████▊  | 78/100 [00:01<00:00, 62.63it/s]


Early stopping
Fold 9 Accuracy: 1.0 MCC: 1.0


Epochs:  55%|█████▌    | 55/100 [00:01<00:01, 43.86it/s]


Early stopping
Fold 10 Accuracy: 0.9310344827586207 MCC: 0.8696565534786727
Elapsed time: 13.29 seconds


In [29]:

print("\n---------------------------------------------------------------------\n")
XII_100_nopcc_results_df = train_with_base_deeptlf(datasets, 'XII_100_nopcc')


---------------------------------------------------------------------

Currently training base DeepTLF model with dataset key: XII_100_nopcc


Epochs:  74%|███████▍  | 74/100 [00:01<00:00, 38.58it/s]


Early stopping
Fold 1 Accuracy: 1.0 MCC: 1.0


Epochs:  45%|████▌     | 45/100 [00:01<00:01, 29.60it/s]


Early stopping
Fold 2 Accuracy: 0.95 MCC: 0.9045340337332909


...

Early stopping
Fold 8 Accuracy: 0.7 MCC: 0.5


Epochs:  52%|█████▏    | 52/100 [00:01<00:01, 28.14it/s]


Early stopping
Fold 9 Accuracy: 0.95 MCC: 0.9045340337332909


Epochs:  73%|███████▎  | 73/100 [00:01<00:00, 39.34it/s]


Early stopping
Fold 10 Accuracy: 0.8421052631578947 MCC: 0.6854365268376295
Elapsed time: 29.69 seconds


In [30]:

print("\n---------------------------------------------------------------------\n")
XII_100_pcc95_results_df = train_with_base_deeptlf(datasets, 'XII_100_pcc95')


---------------------------------------------------------------------

Currently training base DeepTLF model with dataset key: XII_100_pcc95


Epochs:  53%|█████▎    | 53/100 [00:01<00:01, 45.58it/s]


Early stopping
Fold 1 Accuracy: 1.0 MCC: 1.0


Epochs:  70%|███████   | 70/100 [00:01<00:00, 41.17it/s]


Early stopping
Fold 2 Accuracy: 1.0 MCC: 1.0


...

Early stopping
Fold 8 Accuracy: 0.8 MCC: 0.6


Epochs:  50%|█████     | 50/100 [00:01<00:01, 31.28it/s]


Early stopping
Fold 9 Accuracy: 0.9 MCC: 0.8


Epochs:  42%|████▏     | 42/100 [00:00<00:01, 48.94it/s]

Early stopping
Fold 10 Accuracy: 0.8947368421052632 MCC: 0.7888888888888889
Elapsed time: 17.76 seconds


In [31]:

print("\n---------------------------------------------------------------------\n")
XII_100_pcc75_results_df = train_with_base_deeptlf(datasets, 'XII_100_pcc75')


---------------------------------------------------------------------

Currently training base DeepTLF model with dataset key: XII_100_pcc75


Epochs:  52%|█████▏    | 52/100 [00:00<00:00, 63.49it/s]


Early stopping
Fold 1 Accuracy: 1.0 MCC: 1.0


Epochs:  44%|████▍     | 44/100 [00:01<00:01, 39.00it/s]


Early stopping
Fold 2 Accuracy: 1.0 MCC: 1.0


...

Early stopping
Fold 8 Accuracy: 0.9 MCC: 0.816496580927726


Epochs:  54%|█████▍    | 54/100 [00:00<00:00, 63.68it/s]


Early stopping
Fold 9 Accuracy: 0.95 MCC: 0.9045340337332909


Epochs:  72%|███████▏  | 72/100 [00:01<00:00, 56.25it/s]

Early stopping
Fold 10 Accuracy: 0.8947368421052632 MCC: 0.8050764858994133
Elapsed time: 12.23 seconds


In [32]:
print("Base model:")
print('\n---------------------------------------------------------------------\n')
view_results_df(XII_all_nopcc_results_df, 'XII_all_nopcc')
print('\n---------------------------------------------------------------------\n')
view_results_df(XII_all_pcc95_results_df, 'XII_all_pcc95')
print('\n---------------------------------------------------------------------\n')
view_results_df(XII_all_nopcc_results_df, 'XII_all_pcc75')
print('\n---------------------------------------------------------------------\n')
view_results_df(XII_200_nopcc_results_df, 'XII_200_nopcc')
print('\n---------------------------------------------------------------------\n')
view_results_df(XII_200_pcc95_results_df, 'XII_200_pcc95')
print('\n---------------------------------------------------------------------\n')
view_results_df(XII_200_nopcc_results_df, 'XII_200_pcc75')
print('\n---------------------------------------------------------------------\n')
view_results_df(XII_150_nopcc_results_df, 'XII_150_nopcc')
print('\n---------------------------------------------------------------------\n')
view_results_df(XII_150_pcc95_results_df, 'XII_150_pcc95')
print('\n---------------------------------------------------------------------\n')
view_results_df(XII_150_nopcc_results_df, 'XII_150_pcc75')
print('\n---------------------------------------------------------------------\n')
view_results_df(XII_100_nopcc_results_df, 'XII_100_nopcc')
print('\n---------------------------------------------------------------------\n')
view_results_df(XII_100_pcc95_results_df, 'XII_100_pcc95')
print('\n---------------------------------------------------------------------\n')
view_results_df(XII_100_nopcc_results_df, 'XII_100_pcc75')
print('\n---------------------------------------------------------------------\n')


Base model:

---------------------------------------------------------------------

10-fold cross validation results for results_df with key: XII_all_nopcc
Fold 0, Accuracy: 0.878, MCC: 0.765
Fold 1, Accuracy: 0.898, MCC: 0.797
Fold 2, Accuracy: 0.959, MCC: 0.922
Fold 3, Accuracy: 0.898, MCC: 0.797
Fold 4, Accuracy: 0.939, MCC: 0.878
Fold 5, Accuracy: 0.918, MCC: 0.837
Fold 6, Accuracy: 0.878, MCC: 0.757
Fold 7, Accuracy: 0.875, MCC: 0.753
Fold 8, Accuracy: 0.812, MCC: 0.626
Fold 9, Accuracy: 0.792, MCC: 0.585

---------------------------------------------------------------------

10-fold cross validation results for results_df with key: XII_all_pcc95
Fold 0, Accuracy: 0.898, MCC: 0.797
Fold 1, Accuracy: 0.878, MCC: 0.757
Fold 2, Accuracy: 1.000, MCC: 1.000
Fold 3, Accuracy: 0.939, MCC: 0.878
Fold 4, Accuracy: 0.918, MCC: 0.837
Fold 5, Accuracy: 0.878, MCC: 0.757
Fold 6, Accuracy: 0.816, MCC: 0.633
Fold 7, Accuracy: 0.875, MCC: 0.750
Fold 8, Accuracy: 0.875, MCC: 0.750
Fold 9, Accuracy

In [33]:
XII_all_nopcc_results_df = train_with_best_hyperparameters(datasets, 'XII_all_nopcc')


[I 2025-01-04 19:39:34,676] A new study created in memory with name: no-name-d24d78b6-5623-45a9-a569-8602c06bbe76


Currently finetuning DeepTLF model with dataset key: XII_all_nopcc


Epochs:  33%|███▎      | 33/100 [00:01<00:02, 29.10it/s]
[I 2025-01-04 19:39:37,273] Trial 0 finished with value: 0.6327220837212335 and parameters: {'n_est': 15, 'max_depth': 5, 'drop': 0.2, 'hidden_dim': 128, 'n_layers': 6, 'xgb_lr': 0.13}. Best is trial 0 with value: 0.6327220837212335.


Early stopping


Epochs:  30%|███       | 30/100 [00:06<00:14,  4.91it/s]


...

Early stopping
Fold 8 Accuracy: 0.9375 MCC: 0.8757605390397141


Epochs:  36%|███▌      | 36/100 [00:01<00:01, 32.77it/s]


Early stopping
Fold 9 Accuracy: 0.8333333333333334 MCC: 0.6885303726590964


Epochs:  48%|████▊     | 48/100 [00:01<00:01, 38.13it/s]

Early stopping
Fold 10 Accuracy: 0.875 MCC: 0.7526178090063816
Elapsed time: 128.83 seconds


In [34]:
XII_all_pcc95_results_df = train_with_best_hyperparameters(datasets, 'XII_all_pcc95')


[I 2025-01-04 19:41:43,514] A new study created in memory with name: no-name-cdcafe2d-4026-47d1-965a-50eb8726c91e


Currently finetuning DeepTLF model with dataset key: XII_all_pcc95


Epochs:  33%|███▎      | 33/100 [00:02<00:06, 11.15it/s]
[I 2025-01-04 19:41:49,577] Trial 0 finished with value: 0.7145252027247703 and parameters: {'n_est': 45, 'max_depth': 8, 'drop': 0.2, 'hidden_dim': 128, 'n_layers': 2, 'xgb_lr': 0.42000000000000004}. Best is trial 0 with value: 0.7145252027247703.


Early stopping


Epochs:  39%|███▉      | 39/100 [00:01<00:02, 21.42it/s]
[I 2025-01-04 19:41:53,241] Trial 1 finished with value: 0.7145252027247703 and parameters: {'n_est': 35, 'max_depth': 4, 'drop': 0.35, 'hidden_dim': 128, 'n_layers': 4, 'xgb_lr': 0.09}. Best is trial 0 with value: 0.7145252027247703.


...

Early stopping
Fold 8 Accuracy: 0.8958333333333334 MCC: 0.7979249985750353


Epochs:  62%|██████▏   | 62/100 [00:05<00:03, 11.01it/s]


Early stopping
Fold 9 Accuracy: 0.7916666666666666 MCC: 0.5916079783099616


Epochs:  53%|█████▎    | 53/100 [00:05<00:04, 10.59it/s]


Early stopping
Fold 10 Accuracy: 0.7916666666666666 MCC: 0.5916079783099616
Elapsed time: 166.51 seconds


In [35]:
XII_all_pcc75_results_df = train_with_best_hyperparameters(datasets, 'XII_all_pcc75')


[I 2025-01-04 19:44:30,035] A new study created in memory with name: no-name-19a65560-33e4-4c72-b3e6-dc270223779f


Currently finetuning DeepTLF model with dataset key: XII_all_pcc75


Epochs:  49%|████▉     | 49/100 [00:00<00:00, 51.11it/s]
[I 2025-01-04 19:44:31,416] Trial 0 finished with value: 0.7963283217283071 and parameters: {'n_est': 20, 'max_depth': 3, 'drop': 0.45000000000000007, 'hidden_dim': 128, 'n_layers': 4, 'xgb_lr': 0.31}. Best is trial 0 with value: 0.7963283217283071.


Early stopping


Epochs:  41%|████      | 41/100 [00:01<00:02, 29.46it/s]
[I 2025-01-04 19:44:33,788] Trial 1 finished with value: 0.8013876853447538 and parameters: {'n_est': 35, 'max_depth': 5, 'drop': 0.45000000000000007, 'hidden_dim': 192, 'n_layers': 5, 'xgb_lr': 0.19}. Best is trial 1 with value: 0.8013876853447538.


...

Early stopping
Fold 8 Accuracy: 0.8958333333333334 MCC: 0.7979249985750353


Epochs:  29%|██▉       | 29/100 [00:00<00:01, 41.05it/s]


Early stopping
Fold 9 Accuracy: 0.8541666666666666 MCC: 0.7242243488319186


Epochs:  20%|██        | 20/100 [00:00<00:01, 47.78it/s]


Early stopping
Fold 10 Accuracy: 0.5416666666666666 MCC: 0.08362420100070908
Elapsed time: 45.82 seconds


In [36]:
XII_200_nopcc_results_df = train_with_best_hyperparameters(datasets, 'XII_200_nopcc')


[I 2025-01-04 19:45:15,867] A new study created in memory with name: no-name-db673393-2f5f-4e9a-b6e5-166985a825b5


Currently finetuning DeepTLF model with dataset key: XII_200_nopcc


Epochs:  56%|█████▌    | 56/100 [00:02<00:01, 27.41it/s]
[I 2025-01-04 19:45:19,509] Trial 0 finished with value: 0.8 and parameters: {'n_est': 10, 'max_depth': 9, 'drop': 0.30000000000000004, 'hidden_dim': 384, 'n_layers': 3, 'xgb_lr': 0.45}. Best is trial 0 with value: 0.8.


Early stopping


Epochs:  74%|███████▍  | 74/100 [00:08<00:02,  9.03it/s]
[I 2025-01-04 19:45:33,169] Trial 1 finished with value: 0.7585826061362605 and parameters: {'n_est': 35, 'max_depth': 7, 'drop': 0.35, 'hidden_dim': 64, 'n_layers': 4, 'xgb_lr': 0.5}. Best is trial 0 with value: 0.8.


...

Early stopping
Fold 8 Accuracy: 0.875 MCC: 0.7509392614826383


Epochs:  58%|█████▊    | 58/100 [00:06<00:04,  8.75it/s]


Early stopping
Fold 9 Accuracy: 0.95 MCC: 0.9045340337332909


Epochs:  35%|███▌      | 35/100 [00:04<00:07,  8.19it/s]


Early stopping
Fold 10 Accuracy: 0.8461538461538461 MCC: 0.696572345547434
Elapsed time: 271.66 seconds


In [37]:
XII_200_pcc95_results_df = train_with_best_hyperparameters(datasets, 'XII_200_pcc95')


[I 2025-01-04 19:49:47,538] A new study created in memory with name: no-name-13fb76ef-99e5-4c78-a123-6d0031fa76ed


Currently finetuning DeepTLF model with dataset key: XII_200_pcc95


Epochs:  48%|████▊     | 48/100 [00:03<00:03, 13.55it/s]
[I 2025-01-04 19:49:52,793] Trial 0 finished with value: 0.5871365639519862 and parameters: {'n_est': 35, 'max_depth': 5, 'drop': 0.4, 'hidden_dim': 128, 'n_layers': 5, 'xgb_lr': 0.5}. Best is trial 0 with value: 0.5871365639519862.


Early stopping


Epochs:  74%|███████▍  | 74/100 [00:03<00:01, 19.83it/s]
[I 2025-01-04 19:49:58,096] Trial 1 finished with value: 0.9 and parameters: {'n_est': 35, 'max_depth': 4, 'drop': 0.4, 'hidden_dim': 256, 'n_layers': 2, 'xgb_lr': 0.27}. Best is trial 1 with value: 0.9.


...

Early stopping
Fold 8 Accuracy: 0.825 MCC: 0.6508140266182866


Epochs:  55%|█████▌    | 55/100 [00:01<00:01, 35.07it/s]


Early stopping
Fold 9 Accuracy: 0.925 MCC: 0.8510644963469901


Epochs:  50%|█████     | 50/100 [00:01<00:01, 33.98it/s]

Early stopping
Fold 10 Accuracy: 0.9230769230769231 MCC: 0.8473684210526315
Elapsed time: 95.56 seconds


In [38]:
XII_200_pcc75_results_df = train_with_best_hyperparameters(datasets, 'XII_200_pcc75')


[I 2025-01-04 19:51:23,109] A new study created in memory with name: no-name-89e3513e-4b21-4b10-aefa-28419df6e2d4


Currently finetuning DeepTLF model with dataset key: XII_200_pcc75


Epochs:  46%|████▌     | 46/100 [00:00<00:01, 48.23it/s]
[I 2025-01-04 19:51:24,607] Trial 0 finished with value: 0.6508140266182866 and parameters: {'n_est': 20, 'max_depth': 8, 'drop': 0.35, 'hidden_dim': 448, 'n_layers': 3, 'xgb_lr': 0.13}. Best is trial 0 with value: 0.6508140266182866.


Early stopping


Epochs:  28%|██▊       | 28/100 [00:00<00:02, 33.66it/s]
[I 2025-01-04 19:51:26,472] Trial 1 finished with value: 0.6 and parameters: {'n_est': 45, 'max_depth': 10, 'drop': 0.5, 'hidden_dim': 448, 'n_layers': 2, 'xgb_lr': 0.02}. Best is trial 0 with value: 0.6508140266182866.


...

Early stopping
Fold 8 Accuracy: 0.875 MCC: 0.7509392614826383


Epochs:  42%|████▏     | 42/100 [00:01<00:01, 34.54it/s]


Early stopping
Fold 9 Accuracy: 0.925 MCC: 0.8510644963469901


Epochs:  47%|████▋     | 47/100 [00:01<00:01, 42.25it/s]

Early stopping
Fold 10 Accuracy: 0.9230769230769231 MCC: 0.8469686474269935
Elapsed time: 53.07 seconds


In [39]:
XII_150_nopcc_results_df = train_with_best_hyperparameters(datasets, 'XII_150_nopcc')


[I 2025-01-04 19:52:16,189] A new study created in memory with name: no-name-31c09827-ed76-46ef-8acc-74cba1df4f1b


Currently finetuning DeepTLF model with dataset key: XII_150_nopcc


Epochs:  51%|█████     | 51/100 [00:04<00:04, 10.58it/s]
[I 2025-01-04 19:52:23,792] Trial 0 finished with value: 0.6681531047810609 and parameters: {'n_est': 30, 'max_depth': 5, 'drop': 0.5, 'hidden_dim': 512, 'n_layers': 6, 'xgb_lr': 0.21000000000000002}. Best is trial 0 with value: 0.6681531047810609.


Early stopping


Epochs:  44%|████▍     | 44/100 [00:01<00:01, 32.99it/s]
[I 2025-01-04 19:52:27,589] Trial 1 finished with value: 0.8017837257372731 and parameters: {'n_est': 35, 'max_depth': 3, 'drop': 0.2, 'hidden_dim': 128, 'n_layers': 5, 'xgb_lr': 0.22}. Best is trial 1 with value: 0.8017837257372731.


...

Early stopping
Fold 8 Accuracy: 0.9666666666666667 MCC: 0.9354143466934853


Epochs:  31%|███       | 31/100 [00:02<00:05, 12.66it/s]


Early stopping
Fold 9 Accuracy: 0.9666666666666667 MCC: 0.9354143466934853


Epochs:  33%|███▎      | 33/100 [00:02<00:05, 11.56it/s]

Early stopping
Fold 10 Accuracy: 0.9655172413793104 MCC: 0.9330250341152236
Elapsed time: 158.85 seconds


In [40]:
XII_150_pcc95_results_df = train_with_best_hyperparameters(datasets, 'XII_150_pcc95')


[I 2025-01-04 19:54:55,049] A new study created in memory with name: no-name-5aa3803b-467b-4452-8307-1a496b21b60f


Currently finetuning DeepTLF model with dataset key: XII_150_pcc95


Epochs:  44%|████▍     | 44/100 [00:04<00:05,  9.41it/s]
[I 2025-01-04 19:55:02,340] Trial 0 finished with value: 0.7333333333333333 and parameters: {'n_est': 40, 'max_depth': 10, 'drop': 0.45000000000000007, 'hidden_dim': 320, 'n_layers': 3, 'xgb_lr': 0.36000000000000004}. Best is trial 0 with value: 0.7333333333333333.


Early stopping


Epochs:  31%|███       | 31/100 [00:01<00:02, 25.86it/s]
[I 2025-01-04 19:55:04,486] Trial 1 finished with value: 0.6225430174794672 and parameters: {'n_est': 20, 'max_depth': 4, 'drop': 0.25, 'hidden_dim': 448, 'n_layers': 5, 'xgb_lr': 0.31}. Best is trial 0 with value: 0.7333333333333333.


...

Fold 8 Accuracy: 1.0 MCC: 1.0


Epochs:  88%|████████▊ | 88/100 [00:05<00:00, 16.80it/s]


Early stopping
Fold 9 Accuracy: 1.0 MCC: 1.0


Epochs:  43%|████▎     | 43/100 [00:02<00:03, 15.95it/s]


Early stopping
Fold 10 Accuracy: 0.896551724137931 MCC: 0.7942674649391135
Elapsed time: 123.63 seconds


In [41]:
XII_150_pcc75_results_df = train_with_best_hyperparameters(datasets, 'XII_150_pcc75')


[I 2025-01-04 19:56:58,690] A new study created in memory with name: no-name-8294f099-fc40-4475-bc69-cef6deb8a0d9


Currently finetuning DeepTLF model with dataset key: XII_150_pcc75


Epochs:  64%|██████▍   | 64/100 [00:01<00:00, 61.66it/s]
[I 2025-01-04 19:57:00,471] Trial 0 finished with value: 0.7333333333333333 and parameters: {'n_est': 35, 'max_depth': 10, 'drop': 0.30000000000000004, 'hidden_dim': 64, 'n_layers': 3, 'xgb_lr': 0.09}. Best is trial 0 with value: 0.7333333333333333.


Early stopping


Epochs:  50%|█████     | 50/100 [00:01<00:01, 41.85it/s]
[I 2025-01-04 19:57:02,699] Trial 1 finished with value: 0.7399400733959437 and parameters: {'n_est': 45, 'max_depth': 5, 'drop': 0.2, 'hidden_dim': 64, 'n_layers': 5, 'xgb_lr': 0.33}. Best is trial 1 with value: 0.7399400733959437.


...

Early stopping
Fold 8 Accuracy: 1.0 MCC: 1.0


Epochs: 100%|██████████| 100/100 [00:01<00:00, 85.94it/s]


Fold 9 Accuracy: 1.0 MCC: 1.0


Epochs:  87%|████████▋ | 87/100 [00:00<00:00, 94.07it/s] 

Early stopping
Fold 10 Accuracy: 0.9310344827586207 MCC: 0.8696565534786727
Elapsed time: 43.97 seconds


In [42]:
XII_100_nopcc_results_df = train_with_best_hyperparameters(datasets, 'XII_100_nopcc')


[I 2025-01-04 19:57:42,670] A new study created in memory with name: no-name-2df91681-9105-4424-9941-bde2169a309e


Currently finetuning DeepTLF model with dataset key: XII_100_nopcc


Epochs: 100%|██████████| 100/100 [00:01<00:00, 65.88it/s]
[I 2025-01-04 19:57:45,046] Trial 0 finished with value: 1.0 and parameters: {'n_est': 15, 'max_depth': 6, 'drop': 0.45000000000000007, 'hidden_dim': 192, 'n_layers': 3, 'xgb_lr': 0.05}. Best is trial 0 with value: 1.0.
Epochs:  67%|██████▋   | 67/100 [00:06<00:03, 10.89it/s]


Early stopping


[I 2025-01-04 19:57:55,702] Trial 1 finished with value: 1.0 and parameters: {'n_est': 40, 'max_depth': 8, 'drop': 0.1, 'hidden_dim': 512, 'n_layers': 3, 'xgb_lr': 0.4}. Best is trial 0 with value: 1.0.
Epochs:  54%|█████▍    | 54/100 [00:01<00:01, 28.17it/s]
[I 2025-01-04 19:57:59,828] Trial 2 finished with value: 1.0 and parameters: {'n_est': 50, 'max_depth': 3, 'drop': 0.2, 'hidden_dim': 256, 'n_layers': 4, 'xgb_lr': 0.03}. Best is trial 0 with value: 1.0.


...

Early stopping
Fold 8 Accuracy: 0.85 MCC: 0.7035264706814485


Epochs:  43%|████▎     | 43/100 [00:00<00:00, 65.50it/s]


Early stopping
Fold 9 Accuracy: 0.75 MCC: 0.5241424183609591


Epochs:  59%|█████▉    | 59/100 [00:00<00:00, 61.15it/s]


Early stopping
Fold 10 Accuracy: 0.8421052631578947 MCC: 0.6854365268376295
Elapsed time: 118.78 seconds


In [43]:
XII_100_pcc95_results_df = train_with_best_hyperparameters(datasets, 'XII_100_pcc95')


[I 2025-01-04 19:59:41,454] A new study created in memory with name: no-name-76297f13-1fbe-4969-9323-db312847cd46


Currently finetuning DeepTLF model with dataset key: XII_100_pcc95


Epochs:  54%|█████▍    | 54/100 [00:01<00:00, 52.36it/s]
[I 2025-01-04 19:59:42,979] Trial 0 finished with value: 1.0 and parameters: {'n_est': 15, 'max_depth': 3, 'drop': 0.30000000000000004, 'hidden_dim': 320, 'n_layers': 5, 'xgb_lr': 0.31}. Best is trial 0 with value: 1.0.


Early stopping


Epochs:  58%|█████▊    | 58/100 [00:01<00:01, 29.36it/s]
[I 2025-01-04 19:59:46,301] Trial 1 finished with value: 1.0 and parameters: {'n_est': 25, 'max_depth': 10, 'drop': 0.15000000000000002, 'hidden_dim': 512, 'n_layers': 4, 'xgb_lr': 0.21000000000000002}. Best is trial 0 with value: 1.0.


...

Early stopping
Fold 8 Accuracy: 0.9 MCC: 0.8


Epochs:  62%|██████▏   | 62/100 [00:01<00:00, 50.65it/s]


Early stopping
Fold 9 Accuracy: 0.9 MCC: 0.8


Epochs:  37%|███▋      | 37/100 [00:00<00:01, 59.41it/s]

Early stopping
Fold 10 Accuracy: 0.9473684210526315 MCC: 0.9
Elapsed time: 60.04 seconds


In [44]:
XII_100_pcc75_results_df = train_with_best_hyperparameters(datasets, 'XII_100_pcc75')

[I 2025-01-04 20:00:41,499] A new study created in memory with name: no-name-6824eb8a-d7aa-4d33-bd02-05dee208e3a1


Currently finetuning DeepTLF model with dataset key: XII_100_pcc75


Epochs:  79%|███████▉  | 79/100 [00:02<00:00, 37.67it/s]
[I 2025-01-04 20:00:44,413] Trial 0 finished with value: 1.0 and parameters: {'n_est': 45, 'max_depth': 9, 'drop': 0.25, 'hidden_dim': 64, 'n_layers': 6, 'xgb_lr': 0.42000000000000004}. Best is trial 0 with value: 1.0.


Early stopping


Epochs:  51%|█████     | 51/100 [00:01<00:01, 34.88it/s]
[I 2025-01-04 20:00:46,899] Trial 1 finished with value: 0.7337993857053428 and parameters: {'n_est': 45, 'max_depth': 6, 'drop': 0.45000000000000007, 'hidden_dim': 192, 'n_layers': 5, 'xgb_lr': 0.27}. Best is trial 0 with value: 1.0.


...

Early stopping
Fold 8 Accuracy: 0.7 MCC: 0.5


Epochs:  63%|██████▎   | 63/100 [00:01<00:00, 45.56it/s]


Early stopping
Fold 9 Accuracy: 0.9 MCC: 0.8


Epochs: 100%|██████████| 100/100 [00:01<00:00, 52.72it/s]


Fold 10 Accuracy: 0.8947368421052632 MCC: 0.7888888888888889
Elapsed time: 60.30 seconds


In [45]:
print("Finetuned")
print('\n---------------------------------------------------------------------\n')
view_results_df(XII_all_nopcc_results_df, 'XII_all_nopcc')
print('\n---------------------------------------------------------------------\n')
view_results_df(XII_all_pcc95_results_df, 'XII_all_pcc95')
print('\n---------------------------------------------------------------------\n')
view_results_df(XII_all_nopcc_results_df, 'XII_all_pcc75')
print('\n---------------------------------------------------------------------\n')
view_results_df(XII_200_nopcc_results_df, 'XII_200_nopcc')
print('\n---------------------------------------------------------------------\n')
view_results_df(XII_200_pcc95_results_df, 'XII_200_pcc95')
print('\n---------------------------------------------------------------------\n')
view_results_df(XII_200_nopcc_results_df, 'XII_200_pcc75')
print('\n---------------------------------------------------------------------\n')
view_results_df(XII_150_nopcc_results_df, 'XII_150_nopcc')
print('\n---------------------------------------------------------------------\n')
view_results_df(XII_150_pcc95_results_df, 'XII_150_pcc95')
print('\n---------------------------------------------------------------------\n')
view_results_df(XII_150_nopcc_results_df, 'XII_150_pcc75')
print('\n---------------------------------------------------------------------\n')
view_results_df(XII_100_nopcc_results_df, 'XII_100_nopcc')
print('\n---------------------------------------------------------------------\n')
view_results_df(XII_100_pcc95_results_df, 'XII_100_pcc95')
print('\n---------------------------------------------------------------------\n')
view_results_df(XII_100_nopcc_results_df, 'XII_100_pcc75')
print('\n---------------------------------------------------------------------\n')


Finetuned

---------------------------------------------------------------------

10-fold cross validation results for results_df with key: XII_all_nopcc
Fold 0, Accuracy: 0.857, MCC: 0.720
Fold 1, Accuracy: 0.898, MCC: 0.796
Fold 2, Accuracy: 0.959, MCC: 0.922
Fold 3, Accuracy: 0.898, MCC: 0.796
Fold 4, Accuracy: 0.918, MCC: 0.839
Fold 5, Accuracy: 0.898, MCC: 0.797
Fold 6, Accuracy: 0.898, MCC: 0.797
Fold 7, Accuracy: 0.938, MCC: 0.876
Fold 8, Accuracy: 0.833, MCC: 0.689
Fold 9, Accuracy: 0.875, MCC: 0.753

---------------------------------------------------------------------

10-fold cross validation results for results_df with key: XII_all_pcc95
Fold 0, Accuracy: 0.898, MCC: 0.796
Fold 1, Accuracy: 0.796, MCC: 0.608
Fold 2, Accuracy: 1.000, MCC: 1.000
Fold 3, Accuracy: 0.898, MCC: 0.796
Fold 4, Accuracy: 0.878, MCC: 0.757
Fold 5, Accuracy: 0.796, MCC: 0.592
Fold 6, Accuracy: 0.878, MCC: 0.765
Fold 7, Accuracy: 0.896, MCC: 0.798
Fold 8, Accuracy: 0.792, MCC: 0.592
Fold 9, Accuracy: 